# Daily Rainfall Prediction - Global (All 32 Cities)

This notebook trains a Transformer model to predict daily rainfall using data from all 32 cities worldwide.

**Model:** Past 30 days → Next day's rainfall

**Cities:** All 32 cities in the dataset

## 1. Imports

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Dataset
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

## 2. Dataset Classes

In [ ]:
class DailyWeatherDataset(Dataset):
    """读取日级天气数据，可以筛选特定城市或使用所有城市"""

    def __init__(self, csv_path="./dataset_global/weather_daily_global.csv", cities=None, date_col="time"):
        if not os.path.exists(csv_path):
            raise FileNotFoundError(f"❌ 找不到日级数据文件: {csv_path}")

        df = pd.read_csv(csv_path)

        # 筛选城市（如果 cities=None，则使用所有城市）
        if cities is not None:
            df = df[df["city"].isin(cities)].copy()
        else:
            df = df.copy()

        df["date"] = pd.to_datetime(df[date_col])
        df = df.sort_values(["city", "date"]).reset_index(drop=True)

        target_col = "precipitation_sum_mm"
        feature_cols = ["temp_mean_C", "temp_max_C", "temp_min_C", "rh_mean_pct",
                       "press_mean_hPa", "wind_mean_ms", "wind_dir_deg", "cloud_mean_pct",
                       "dew_point_C", "month", "month_sin", "month_cos",
                       "precip_lag_1", "precip_lag_3", "temp_mean_lag_1", "precip_roll7",
                       "lat", "lon"]

        # 清洗 NaN/Inf
        cols_to_check = feature_cols + [target_col]
        df[cols_to_check] = df[cols_to_check].replace([np.inf, -np.inf], np.nan)
        before = len(df)
        df = df.dropna(subset=cols_to_check).reset_index(drop=True)
        print(f"🧹 已去除含 NaN/Inf 的行: {before - len(df)} 行，剩余 {len(df)} 行")

        self.feature_cols = feature_cols
        self.target_col = target_col

        # 标准化
        self.mean = df[feature_cols].mean()
        self.std = df[feature_cols].std().replace(0, 1e-6)
        df[feature_cols] = (df[feature_cols] - self.mean) / (self.std + 1e-6)
        self.df = df.reset_index(drop=True)

        print(f"✅ 城市: {sorted(self.df['city'].unique())}")
        print(f"✅ 总天数: {len(self.df)}, 特征维度: {len(self.feature_cols)}")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        x = torch.tensor(row[self.feature_cols].astype(float).values, dtype=torch.float32)
        y = torch.tensor(float(row[self.target_col]), dtype=torch.float32)
        return x, y, row["city"], row["date"]

In [ ]:
class DailySequenceDataset(Dataset):
    """把 DailyWeatherDataset 转成序列：过去 lookback 天 → 明天 (log1p)"""

    def __init__(self, daily_ds, lookback=30):
        self.daily_ds = daily_ds
        self.feature_cols = daily_ds.feature_cols
        self.target_col = daily_ds.target_col
        self.lookback = lookback
        self.df = daily_ds.df
        self.samples = []

        for city, df_city in self.df.groupby("city"):
            idxs = df_city.index.to_list()
            if len(idxs) <= lookback:
                continue
            for i in range(lookback, len(idxs)):
                self.samples.append((city, idxs[i - lookback], idxs[i]))

        print(f"✅ 序列样本数: {len(self.samples)}, lookback = {lookback} 天")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        city, start, end = self.samples[idx]
        window = self.df.loc[start:end-1, self.feature_cols].values.astype("float32")
        target = float(self.df.loc[end, self.target_col])
        x = torch.tensor(window, dtype=torch.float32)
        y_log = torch.tensor(np.log1p(target), dtype=torch.float32)
        return x, y_log

## 3. Device Selection

In [ ]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
    print(f"🧠 使用设备: {device}")
    return device

device = get_device()

## 4. Model Architecture

In [ ]:
class RainfallTransformer(nn.Module):
    def __init__(self, input_dim, seq_len=30, d_model=128, nhead=4, num_layers=3, dropout=0.1):
        super().__init__()
        self.input_dim = input_dim
        self.seq_len = seq_len
        self.input_proj = nn.Linear(input_dim, d_model)
        self.pos_embedding = nn.Parameter(torch.randn(1, seq_len, d_model))
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=4*d_model,
            dropout=dropout, activation="gelu", batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.head = nn.Sequential(
            nn.LayerNorm(d_model), nn.Linear(d_model, 128), nn.GELU(),
            nn.Dropout(dropout), nn.Linear(128, 1))

    def forward(self, x):
        if x.size(1) > self.seq_len:
            x = x[:, -self.seq_len:, :]
        x_proj = self.input_proj(x) + self.pos_embedding[:, :x.size(1)]
        enc = self.encoder(x_proj)
        pooled = enc.mean(dim=1)
        return self.head(pooled).squeeze(-1)

## 5. Training Functions

In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    for xb, yb in tqdm(loader, desc="👟 训练"):
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        total_loss += loss.item() * xb.size(0)
    return total_loss / len(loader.dataset)

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    all_pred_log, all_true_log = [], []
    with torch.no_grad():
        for xb, yb in tqdm(loader, desc="🔎 验证"):
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss = criterion(pred, yb)
            total_loss += loss.item() * xb.size(0)
            all_pred_log.append(pred.cpu())
            all_true_log.append(yb.cpu())
    avg_loss = total_loss / len(loader.dataset)
    pred_log = torch.cat(all_pred_log).numpy()
    true_log = torch.cat(all_true_log).numpy()
    rmse_log = float(np.sqrt(np.mean((pred_log - true_log) ** 2)))
    pred_mm = np.expm1(pred_log)
    true_mm = np.expm1(true_log)
    rmse_mm = float(np.sqrt(np.mean((pred_mm - true_mm) ** 2)))
    mae_mm = float(np.mean(np.abs(pred_mm - true_mm)))
    return avg_loss, rmse_log, rmse_mm, mae_mm

## 6. Load Data - ALL Cities

In [ ]:
daily_ds = DailyWeatherDataset(
    csv_path="./dataset_global/weather_daily_global.csv",
    cities=None,  # None = all cities
    date_col="time")

print(f"\n📊 城市数量: {daily_ds.df['city'].nunique()}")
print(f"总样本数: {len(daily_ds)}")

seq_ds = DailySequenceDataset(daily_ds, lookback=30)

## 6.5. Print Normalization Statistics

In [ ]:
# Print normalization statistics for use in production
print("\n" + "="*80)
print("📊 NORMALIZATION STATISTICS (Copy these to weather_service.py)")
print("="*80)
print("\nFeature columns:")
for i, col in enumerate(daily_ds.feature_cols):
    print(f"  {i}: {col}")

print("\nDAILY_FEATURE_MEAN = np.array([")
for i, (col, val) in enumerate(zip(daily_ds.feature_cols, daily_ds.mean)):
    print(f"    {val:.6f},  # {col}")
print("])")

print("\nDAILY_FEATURE_STD = np.array([")
for i, (col, val) in enumerate(zip(daily_ds.feature_cols, daily_ds.std)):
    print(f"    {val:.6f},  # {col}")
print("])")
print("\n" + "="*80 + "\n")

## 7. Train/Val Split

In [ ]:
val_size = int(len(seq_ds) * 0.2)
train_size = len(seq_ds) - val_size
train_ds, val_ds = random_split(seq_ds, [train_size, val_size],
                                generator=torch.Generator().manual_seed(42))
print(f"训练集: {train_size}, 验证集: {val_size}")

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)

## 8. Initialize Model

In [ ]:
model = RainfallTransformer(input_dim=len(daily_ds.feature_cols), seq_len=30).to(device)
criterion = nn.HuberLoss(delta=0.1)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-3)

total_params = sum(p.numel() for p in model.parameters())
print(f"🤖 总参数: {total_params:,}")

## 9. Training Loop

In [ ]:
num_epochs = 15
best_val_loss = float("inf")
best_path = "daily_transformer_global_best.pt"
train_losses, val_losses = [], []

for epoch in range(1, num_epochs + 1):
    print(f"\n===== Epoch {epoch}/{num_epochs} =====")
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    print(f"📉 训练损失: {train_loss:.6f}")
    val_loss, rmse_log, rmse_mm, mae_mm = evaluate(model, val_loader, criterion, device)
    print(f"✅ 验证损失: {val_loss:.6f}, RMSE(mm): {rmse_mm:.4f}, MAE(mm): {mae_mm:.4f}")
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), best_path)
        print(f"💾 已保存最优模型: {best_path}")

print(f"\n🎉 训练完成！最佳验证损失: {best_val_loss:.6f}")

## 10. Plot Results

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_losses)+1), train_losses, label="Train", marker="o")
plt.plot(range(1, len(val_losses)+1), val_losses, label="Val", marker="s")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training & Validation Loss (All 32 Cities)")
plt.legend()
plt.grid(True)
plt.savefig("loss_curve_global.png", dpi=150)
plt.show()
print("📈 已保存 loss_curve_global.png")